In [ ]:
cd /content/drive/MyDrive/fruit360

/content/drive/MyDrive/fruit360


In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
img_rows, img_cols = 100,100
batch_size = 32
#num_classes = 131
 
train_data_dir = '/content/drive/MyDrive/fruit360/fruits-360/Training'
validation_data_dir = '/content/drive/MyDrive/fruit360/fruits-360/Test'
 
#Lets use some data augmentation
train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=30,
                width_shift_range=0.3,
                height_shift_range=0.3,
                horizontal_flip=True,
                fill_mode='nearest'
                )
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size = (img_rows,img_cols),
                    batch_size=batch_size,
                    class_mode = 'categorical',
                    shuffle= True)
 
validation_generator = validation_datagen.flow_from_directory(
                    validation_data_dir,
                    target_size = (img_rows,img_cols),
                    batch_size=batch_size,
                    class_mode = 'categorical',
                    shuffle= False)

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [ ]:
# Sequential layer banaune
# padding - 'same' results in padding the input such that
# the output has the same length as the original input
num_classes = 131
model = Sequential()
model.add(Conv2D(32,(3,3), padding='same',input_shape=(img_rows, img_cols,3)))
 
model.add(Activation('relu'))
 
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(64,(3,3), padding='same'))
model.add(Activation('relu'))
 
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
 
classifier = Sequential()
classifier.add(Dense(units=512, activation='relu'))
classifier.add(Dense(units=256, activation='relu'))
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=81, activation='relu'))
#classifier.add(Dense(units=32, activation='relu'))
#classifier.add(Dense(units=16, activation='relu'))
#classifier.add(Dense(units=8, activation='relu'))
#classifier.add(Dense(units=4, activation='relu'))
 
model.add(Dense(num_classes))
model.add(Activation('softmax'))
 
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
activation_11 (Activation)   (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 98, 98, 32)        9248      
_________________________________________________________________
activation_12 (Activation)   (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 49, 49, 64)       

In [ ]:
# initiate RMSprop optimizer and configure smoe parameters
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
 
checkpoint = ModelCheckpoint("/content/drive/MyDrive/fruit360/fruits-360/fruit_V1.h5", # yo model ley best weight lai save garcha
                            monitor="val_loss",
                            mode="min",
                            save_best_only=True,
                            verbose=1)
 
earlystop = EarlyStopping(monitor = 'val_loss', #value being monitored for improvement
                         min_delta = 0.001, # Abs value and is the main change required before we stop
                         patience = 3, # no of epocs we wait before stopping
                         verbose = 1,
                         restore_best_weights=True) #keep the best weights once stopped
 
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                           factor = 0.1,
                           patience=3,
                           verbose=1,
                           min_delta=0.0001)
 
# we put our callbacks into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]
 
# We use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
             optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])
 
nb_train_samples = 20000
nb_validation_samples = 600
epochs = 10
 
history = model.fit(
            train_generator,
            steps_per_epoch = nb_train_samples//batch_size,
            epochs = epochs,
            callbacks = callbacks,
            validation_data = validation_generator,
            validation_steps = nb_validation_samples//batch_size)
model.save("/content/drive/MyDrive/fruit360/fruits-360/fruit_V2.h5") # yo model ley sabai lai save garcha

Epoch 1/10
625/625 [==============================] - 1060s 2s/step - loss: 4.2659 - accuracy: 0.0717 - val_loss: 2.5440 - val_accuracy: 0.1753

Epoch 00001: val_loss improved from inf to 2.54400, saving model to /content/drive/MyDrive/fruit360/fruits-360/fruit_V1.h5
Epoch 2/10
625/625 [==============================] - 1028s 2s/step - loss: 1.6800 - accuracy: 0.4814 - val_loss: 1.0788 - val_accuracy: 0.7500

Epoch 00002: val_loss improved from 2.54400 to 1.07883, saving model to /content/drive/MyDrive/fruit360/fruits-360/fruit_V1.h5
Epoch 3/10
625/625 [==============================] - 1018s 2s/step - loss: 0.9786 - accuracy: 0.7042 - val_loss: 0.3202 - val_accuracy: 0.9306

Epoch 00003: val_loss improved from 1.07883 to 0.32017, saving model to /content/drive/MyDrive/fruit360/fruits-360/fruit_V1.h5
Epoch 4/10
625/625 [==============================] - 1014s 2s/step - loss: 0.6958 - accuracy: 0.7816 - val_loss: 0.5355 - val_accuracy: 0.8212

Epoch 00004: val_loss did not improve from 